<a href="https://colab.research.google.com/github/Ditsuhi/ConvLSTM_Madrid/blob/main/NO2_prediction_pandemic_and_non_pandemic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

def CalcIDWvalue(array_interpolate):
  unknown_values = []
  weighted_values_sum = 0.0
  sum_of_weights = 0.0
  sensitivity = 2
  eps = 1e-12
  neg_half_sens = -sensitivity/2.0

  array_float = array_interpolate.astype(float)
  knowncell_position= np.argwhere(array_float!=0)  
  knowncell_value = array_float[array_float!=0]  
  unknowncell_position = np.argwhere(array_float==0) 

  weight = (((knowncell_position[:,0][None,:]-unknowncell_position[:,0][:,None])**2 + (knowncell_position[:,1][None,:]-unknowncell_position[:,1][:,None])**2) + eps)**neg_half_sens
  
  unknown_values = np.sum(knowncell_value[None,:] * weight, axis=1) / np.sum(weight, axis=1)
  array_float[array_float == 0 ] = unknown_values
  return array_float.tolist()



In [ ]:
def calc_idw_fullData(full_data):
  idw_list =[]
  for item in full_data:
    idw_list.append(CalcIDWvalue(item))

  return idw_list


In [ ]:
def calculate_idw_fullData_allAttributes (df_all):
  df_all_idw_list = []
  for attr_numb in range(df_all.shape[2]):
    certain_attr = df_all[:, :, attr_numb]
    certain_attr_reshaped= certain_attr.reshape(certain_attr.shape[0], 20, 17)
    certain_attr_reshaped_idw = calc_idw_fullData(certain_attr_reshaped)    
    certain_attr_reshaped_idw_original_shape = np.reshape(certain_attr_reshaped_idw, (certain_attr.shape[0], 340))

    df_all_idw_list.append(certain_attr_reshaped_idw_original_shape.tolist())
  
  
  df_all_idw_array = np.dstack((item)for item in df_all_idw_list)
  return df_all_idw_array


In [ ]:
import zipfile
from glob import glob
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error




#unzip data giving the path of certain dataset
path = '/content/Jan_Jun_2020.zip'
# path = '/content/Jan_Jun_2019.zip'


with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('/content/')


airMet = glob("/content/content/csvFiles/*.csv")


In [ ]:
#sort dataset with chronological order
def sortingFiles(eachFile):
    return int(eachFile) if eachFile.isdigit() else eachFile
def natural_keys(eachFile):
    return [sortingFiles(c) for c in re.split('(\d+)',eachFile)]

sorted_airMet= sorted(airMet, key = natural_keys)

In [ ]:
# df = [pd.read_csv(f, usecols=[' NO2']).values for f in sorted_airMet]
df = [pd.read_csv(f, usecols=[' NO2', ' windSpeed', ' windDir', ' Temp', ' Humidity', ' Pressure', ' SolarRad', ' Prec']).values for f in sorted_airMet]

df_all  = np.asarray(df)

In [ ]:
# handle outliers

res = np.where(df_all[:, :, 3] < -3)
reshum = np.where(df_all[:, :, 4] < 0)

print(len(res[1]))
print(len(reshum[1]))

0
0


In [ ]:

for i in range(len(res[0])):
  if df_all[:, :, 3][res[0][i]][res[1][i]-1] > -3 and df_all[:, :, 3][res[0][i]][res[1][i]+1] > -3:
    df_all[:, :, 3][res[0][i]][res[1][i]] = (df_all[:, :, 3][res[0][i]][res[1][i]-1]+df_all[:, :, 3][res[0][i]][res[1][i]+1])/2


for i in range(len(reshum[0])):
  if df_all[:, :, 4][reshum[0][i]][reshum[1][i]-1] >= 0 and df_all[:, :, 4][reshum[0][i]][reshum[1][i]+1] >= 0:
    df_all[:, :, 4][reshum[0][i]][reshum[1][i]] = (df_all[:, :, 4][reshum[0][i]][reshum[1][i]-1]+df_all[:, :, 4][reshum[0][i]][reshum[1][i]+1])/2

In [ ]:
df_all_non_prec = np.delete(df_all, 7, 2)

In [ ]:

idw_dataframe = calculate_idw_fullData_allAttributes (df_all_non_prec)
not_nun = np.nan_to_num(idw_dataframe)
round_data = np.round(not_nun, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


In [ ]:
 
# split dataset to X and y (dependent and independent)
def split_sequence(sequence, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
   
		# find the end of this pattern
		end_ix = i + 12
    
		# check if we are beyond the sequence
		if end_ix+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern
    # seq_x, seq_y = sequence[i:end_ix], sequence[end_ix: end_ix+time_steps]
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix+time_steps]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 

 # define input sequence
raw_seq = round_data

# time_steps = 6
time_steps = 12

X, y_total = split_sequence(raw_seq, time_steps)
#to take only NO2 from whole features as a target variable
y =y_total[:, :, 0]


In [ ]:
#split data to train and test sets

X_train_notNorm, X_test_notNorm, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)

In [ ]:
# to normalise train data using MinMaxScaler
number_selected_columns = 7

scaler = MinMaxScaler(feature_range=(0, 1), copy = False)
train_Normalised = X_train_notNorm.reshape(-1, 340*number_selected_columns)
test_Normalised = X_test_notNorm.reshape(-1, 340*number_selected_columns)

train_scaled = scaler.fit_transform(train_Normalised)
test_scaled = scaler.transform(test_Normalised)

X_train = train_scaled.reshape(X_train_notNorm.shape[0], X_train_notNorm.shape[1], X_train_notNorm.shape[2], X_train_notNorm.shape[3])
X_test = test_scaled.reshape(X_test_notNorm.shape[0], X_test_notNorm.shape[1], X_test_notNorm.shape[2], X_test_notNorm.shape[3])

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 20, 17*number_selected_columns, 1))
y_train_reshaped = y_train.reshape((y_train.shape[0], 20, 17*number_selected_columns, 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 20, 17*number_selected_columns, 1))
y_test_reshaped = y_test.reshape(y_test.shape[0], 20, 17*number_selected_columns, 1)

In [ ]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [ ]:

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


btscv = BlockingTimeSeriesSplit(n_splits=3)
model = KerasRegressor(build_fn=create_model, verbose=0)


#define the grid search parameters

init_mode = ['uniform', 'normal',  'glorot_normal', 'glorot_uniform']
optimizer = ['RMSprop',  'Adam']
kernel_size = [(3, 3), (5, 5), (7, 7), (3, 1),(5, 1), (10,1)]
dropout_rate = [ 0.2, 0.3,  0.5],
filters= [8, 16, 32, 64]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
param_grid = dict(filters=filters, dropout_rate=dropout_rate, kernel_size=kernel_size, optimizer=optimizer, init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=btscv)
grid_result = grid.fit(X_train_reshaped, y_train_reshaped, epochs = 20)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Dropout, BatchNormalization
from  keras.regularizers import l2


def create_model(number_selected_columns=7, optimizer='adam', kernel_size=(5, 1), filters=filters, dropout_rate=0.2, init_mode="glorot_uniform"):
    
    model = Sequential()
    
    model.add(ConvLSTM2D(input_shape=(None,  340, number_selected_columns, 1),  filters=filters, kernel_initializer=init_mode,  kernel_size=kernel_size, padding="same", return_sequences=True, kernel_regularizer=l2(0.01)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())    
    model.add(ConvLSTM2D(filters=filters, kernel_initializer=init_mode,  kernel_size=kernel_size, padding="same", return_sequences=True))
    model.add(Dropout(dropout_rate))  
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=filters,  kernel_initializer=init_mode,  kernel_size=kernel_size, padding="same", return_sequences=True))
    model.add(Dropout(dropout_rate))   
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=filters, kernel_initializer=init_mode,  kernel_size=kernel_size, padding="same", return_sequences=True))
    model.add(Dropout(dropout_rate))    
    model.add(BatchNormalization())          
    model.add(ConvLSTM2D(filters=1, kernel_initializer=init_mode, kernel_size=(1, number_selected_columns), activation='relu'))
    model.compile(optimizer=optimizer, loss='mse')
   
    
    print(model.summary())

    return model

In [ ]:
mod = create_model(number_selected_columns=7, optimizer='adam', kernel_size=(5,1), filters=32 dropout_rate=0.2, init_mode="glorot_uniform")

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
history = mod.fit(X_train_reshaped, y_train_reshaped,  epochs=100, verbose=2, validation_split=0.2, shuffle=False, callbacks=[es])

In [ ]:
yhat = mod.predict(X_test_reshaped , verbose=1)
yhat_reshaped = yhat.reshape(y_test.shape[0], 340)

testScore = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
print('Test Score: %.2f RMSE' % (testScore))